In [1]:
# Based on https://blog.floydhub.com/spinning-up-with-deep-reinforcement-learning/
import gym
import random
import time
import os
import pylab as pl
import matplotlib.pyplot as plt
from IPython import display
from datetime import datetime
from keras.layers import Dense
from keras.models import Sequential
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
# Collect a game of Pong with random input.
frames = []
frames_ram = []
STEPS = 300

UP_ACTION = 2
DOWN_ACTION = 3

env = gym.make("Pong-ram-v0")

observation = env.reset()

for i in range(STEPS):
    gym.envs.registry
    action = random.randint(UP_ACTION, DOWN_ACTION)

    observation, reward, done, info = env.step(action)
    frames.append(env.unwrapped._get_image())
    frames_ram.append(env.unwrapped._get_ram())
    
    if done:
        observation = env.reset()
        frames.append(observation)

In [3]:
# Draw our frames to see what happened and print ram to find important addresses.
ram_address_position_player_y = 60
ram_address_position_ball_x = 49
ram_address_position_ball_y = 50

'''
for i in range(len(frames)):
    plt.imshow(frames[i])
    print(frames_ram[i])
    print('Player position y: ' + str(frames_ram[i][ram_address_position_player_y]))
    print('Ball position x: ' + str(frames_ram[i][ram_address_position_ball_x]))
    print('Ball position y: ' + str(frames_ram[i][ram_address_position_ball_y]))
    display.display(pl.gcf())
    time.sleep(0.5)
    display.clear_output(wait=True)
'''


"\nfor i in range(len(frames)):\n    plt.imshow(frames[i])\n    print(frames_ram[i])\n    print('Player position y: ' + str(frames_ram[i][ram_address_position_player_y]))\n    print('Ball position x: ' + str(frames_ram[i][ram_address_position_ball_x]))\n    print('Ball position y: ' + str(frames_ram[i][ram_address_position_ball_y]))\n    display.display(pl.gcf())\n    time.sleep(0.5)\n    display.clear_output(wait=True)\n"

In [4]:
# Create the neural network.
# TODO: What is a adam optimizer?
model = Sequential()
model.add(Dense(units=64,input_dim=6, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(units=64, activation='relu', kernel_initializer='glorot_uniform'))
model.add(Dense(units=1, activation='sigmoid', kernel_initializer='RandomNormal'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
import numpy as np
import gym

# gym initialization
env = gym.make("Pong-ram-v0")
observation = env.reset()
prev_input = None

# Macros
UP_ACTION = 2
DOWN_ACTION = 3

# Hyperparameters
gamma = 0.99

# initialization of variables used in the main loop
x_train, y_train, rewards = [],[],[]
reward_sum = 0
episode_nb = 0

In [6]:
log_dir = './log' + datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
resume = True
epochs_before_saving = 100

# load pre-trained model if exist
if (resume and os.path.isfile('my_model_weights.h5')):
    print("loading previous weights")
    model.load_weights('my_model_weights.h5')

In [7]:
def discount_rewards(r, gamma):
    """ take 1D float array of rewards and compute discounted reward """
    r = np.array(r)
    #print('discount_rewards r:' + str(r))
    discounted_r = np.zeros_like(r)
    running_add = 0
    # we go from last reward to first one so we don't have to do exponentiations
    for t in reversed(range(0, r.size)):
        if r[t] != 0: running_add = 0 # if the game ended (in Pong), reset the reward sum
        running_add = running_add * gamma + r[t] # the point here is to use Horner's method to compute those rewards efficiently
        discounted_r[t] = running_add
    discounted_r -= np.mean(discounted_r) #normalizing the result
    discounted_r /= np.std(discounted_r) #idem
    #print('discount_rewards discounted_r:' + str(discounted_r))
    return discounted_r

In [8]:
# main loop
all_reward_sums = np.array([])
prev_position_player_y = 0
prev_position_ball_x = 0
prev_position_ball_y = 0
while (True):
    ram = env.unwrapped._get_ram()
    position_player_y = int(ram[ram_address_position_player_y])
    position_ball_x = int(ram[ram_address_position_ball_x])
    position_ball_y = int(ram[ram_address_position_ball_y])

    # TODO: Try to give direction as number between 0-1 (0-360°) instead of prev position.
    x = np.array([position_player_y, position_ball_x, position_ball_y, prev_position_player_y, prev_position_ball_x, prev_position_ball_y])
    # print('X: ' + str(x))
    
    prev_position_player_y = position_player_y
    prev_position_ball_x = position_ball_x
    prev_position_ball_y = position_ball_y
    
    # TODO: Why in the simulation it gets stuck at the top? But actually it should regulate itself?
    # Draw current state.
    '''
    plt.imshow(env.unwrapped._get_image())
    print('TV:')
    display.display(pl.gcf())
    time.sleep(0.5)
    display.clear_output(wait=True)
    '''
    
    # forward the policy network and sample action according to the proba distribution
    proba = model.predict(np.array([x]))
    # print('Prediction: ' + str(proba))
    # Variable proba is the probability prediction of how good UP_ACTION is for this frame.
    # Then select UP_ACTION by proba percent. Easy way to still allow the other action.
    # TODO: Mathematical reason for random number?
    action = UP_ACTION if np.random.uniform() < proba else DOWN_ACTION
    y = 1 if action == UP_ACTION else 0 # 0 and 1 are our labels

    # log the input and label to train later
    x_train.append(x)
    y_train.append(y)

    # do one step in our environment
    observation, reward, done, info = env.step(action)
    #print('Observation: ' + str(observation))
    #print('Reward: ' + str(reward))
    #print('Done: ' + str(done))
    #print('Info: ' + str(info))
    rewards.append(reward)
    reward_sum += reward
    
    # end of an episode
    if done:
        print('At the end of episode', episode_nb, 'the total reward was :', reward_sum)
        all_reward_sums = np.append(all_reward_sums, reward_sum)
        print('Average of current run: ' + str(np.average(all_reward_sums)))
        
        # increment episode number
        episode_nb += 1
        
        # training
        # TODO: Is np.vstack is really necessary?
        # TODO: Clarify the use of gamma in sample_weight=discount_rewards(rewards, gamma)
        model.fit(x=np.vstack(x_train), y=np.vstack(y_train), verbose=1, sample_weight=discount_rewards(rewards, gamma))
                                                     
        if episode_nb % epochs_before_saving == 0:    
            model.save_weights('my_model_weights' + datetime.now().strftime("%Y%m%d-%H%M%S") + '.h5')
            
        # Reinitialization
        x_train, y_train, rewards = [],[],[]
        observation = env.reset()
        reward_sum = 0
        prev_input = None

X: [109   0  22   0   0   0]
Prediction: [[0.43105847]]
X: [118   0  16 109   0  22]
Prediction: [[0.6203719]]
X: [128   0  12 118   0  16]
Prediction: [[0.6606671]]
X: [133   0   8 128   0  12]
Prediction: [[0.678141]]
X: [135   0   6 133   0   8]
Prediction: [[0.6905365]]
X: [131   0   0 135   0   6]
Prediction: [[0.68394417]]
X: [122   0   0 131   0   0]
Prediction: [[0.6719895]]
X: [102   0   0 122   0   0]
Prediction: [[0.6216412]]
X: [100   0   0 102   0   0]
Prediction: [[0.6531773]]
X: [ 88   0   0 100   0   0]
Prediction: [[0.6158099]]
X: [97  0  0 88  0  0]
Prediction: [[0.67322224]]
X: [87  0  0 97  0  0]
Prediction: [[0.6181823]]
X: [67  0  0 87  0  0]
Prediction: [[0.5665644]]
X: [56  0  0 67  0  0]
Prediction: [[0.5676914]]
X: [47  0  0 56  0  0]
Prediction: [[0.5573889]]
X: [42  0  0 47  0  0]
Prediction: [[0.55754083]]
X: [39  0  0 42  0  0]
Prediction: [[0.5568166]]
X: [38  0  0 39  0  0]
Prediction: [[0.55940515]]
X: [38  0  0 38  0  0]
Prediction: [[0.5612577]]
X: [3

Prediction: [[0.7206821]]
X: [ 52 154 154  62 152 152]
Prediction: [[0.69214565]]
X: [ 41 157 158  52 154 154]
Prediction: [[0.6299024]]
X: [ 39 159 160  41 157 158]
Prediction: [[0.59522283]]
X: [ 38 162 162  39 159 160]
Prediction: [[0.57663196]]
X: [ 38 165 166  38 162 162]
Prediction: [[0.55711]]
X: [ 38 169 170  38 165 166]
Prediction: [[0.5475585]]
X: [ 38 173 174  38 169 170]
Prediction: [[0.5354329]]
X: [ 38 176 176  38 173 174]
Prediction: [[0.53073627]]
X: [ 38 179 180  38 176 176]
Prediction: [[0.5146234]]
X: [ 38 182 182  38 179 180]
Prediction: [[0.51245874]]
X: [ 38 184 184  38 182 182]
Prediction: [[0.5038056]]
X: [ 40 187 188  38 184 184]
Prediction: [[0.5122856]]
X: [ 47 190 190  40 187 188]
Prediction: [[0.5937408]]
X: [ 50 192 192  47 190 190]
Prediction: [[0.64149046]]
X: [ 52 194 194  50 192 192]
Prediction: [[0.6660869]]
X: [ 68 198 198  52 194 194]
Prediction: [[0.80617774]]
X: [ 69 201 202  68 198 198]
Prediction: [[0.78127164]]
X: [ 51 205 206  69 201 202]
Pred

Prediction: [[0.5973215]]
X: [ 43 153 154  38 151 152]
Prediction: [[0.6435224]]
X: [ 40 156 156  43 153 154]
Prediction: [[0.62512404]]
X: [ 39 158 158  40 156 156]
Prediction: [[0.60029334]]
X: [ 38 160 160  39 158 158]
Prediction: [[0.5800983]]
X: [ 38 163 164  38 160 160]
Prediction: [[0.5631258]]
X: [ 38 167 168  38 163 164]
Prediction: [[0.5536004]]
X: [ 43 169 170  38 167 168]
Prediction: [[0.5975666]]
X: [ 39 173 174  43 169 170]
Prediction: [[0.561128]]
X: [ 38 176 176  39 173 174]
Prediction: [[0.534453]]
X: [ 38 179 180  38 176 176]
Prediction: [[0.5146234]]
X: [ 38 181 182  38 179 180]
Prediction: [[0.50700897]]
X: [ 38 184 184  38 181 182]
Prediction: [[0.5063566]]
X: [ 38 188 188  38 184 184]
Prediction: [[0.495661]]
X: [ 38 191 192  38 188 188]
Prediction: [[0.4780146]]
X: [ 47 195 196  38 191 192]
Prediction: [[0.56728595]]
X: [ 44 197 198  47 195 196]
Prediction: [[0.55775136]]
X: [ 39 201 202  44 197 198]
Prediction: [[0.483275]]
X: [ 38 204 204  39 201 202]
Predictio

X: [ 38 160 160  38 157 158]
Prediction: [[0.57894564]]
X: [ 38 162 162  38 160 160]
Prediction: [[0.57048404]]
X: [ 38 164 164  38 162 162]
Prediction: [[0.5644906]]
X: [ 47 167 168  38 164 164]
Prediction: [[0.6462142]]
X: [ 46 171 172  47 167 168]
Prediction: [[0.654991]]
X: [ 40 175 176  46 171 172]
Prediction: [[0.57337993]]
X: [ 39 178 178  40 175 176]
Prediction: [[0.54307014]]
X: [ 38 181 182  39 178 178]
Prediction: [[0.51225394]]
X: [ 43 185 186  38 181 182]
Prediction: [[0.55385244]]
X: [ 40 188 188  43 185 186]
Prediction: [[0.534837]]
X: [ 38 192 192  40 188 188]
Prediction: [[0.49091965]]
X: [ 38 194 194  38 192 192]
Prediction: [[0.47330773]]
X: [ 38 196 196  38 194 194]
Prediction: [[0.46722478]]
X: [ 38 198 198  38 196 196]
Prediction: [[0.4624032]]
X: [ 38 202 202  38 198 198]
Prediction: [[0.45955417]]
X: [ 38 205 204  38 202 202]
Prediction: [[0.4553799]]
X: [ 47 205 200  38 205 204]
Prediction: [[0.56609994]]
X: [ 56 205 194  47 205 200]
Prediction: [[0.7203558]]
X

Prediction: [[0.57446307]]
X: [ 38 164 164  38 162 162]
Prediction: [[0.5644906]]
X: [ 38 166 166  38 164 164]
Prediction: [[0.5584777]]
X: [ 38 170 170  38 166 166]
Prediction: [[0.5504303]]
X: [ 38 174 174  38 170 170]
Prediction: [[0.5383182]]
X: [ 38 176 176  38 174 174]
Prediction: [[0.52819395]]
X: [ 38 178 178  38 176 176]
Prediction: [[0.52210474]]
X: [ 40 181 182  38 178 178]
Prediction: [[0.53056407]]
X: [ 47 183 184  40 181 182]
Prediction: [[0.6060897]]
X: [ 41 187 188  47 183 184]
Prediction: [[0.5535509]]
X: [ 38 191 192  41 187 188]
Prediction: [[0.49175197]]
X: [ 38 193 194  38 191 192]
Prediction: [[0.4704166]]
X: [ 38 195 196  38 193 194]
Prediction: [[0.46433827]]
X: [ 38 198 198  38 195 196]
Prediction: [[0.46398345]]
X: [ 38 202 202  38 198 198]
Prediction: [[0.45955417]]
X: [ 38 205 204  38 202 202]
Prediction: [[0.4553799]]
X: [ 38 205 202  38 205 204]
Prediction: [[0.45923233]]
X: [ 40 205 196  38 205 202]
Prediction: [[0.5145018]]
X: [ 56 205 188  40 205 196]
P

...

X: [203 205 138 203 205 140]
Prediction: [[3.490079e-10]]
X: [203 205 134 203 205 138]
Prediction: [[5.561441e-10]]
X: [203 205 128 203 205 134]
Prediction: [[1.1166675e-09]]
X: [203 205 124 203 205 128]
Prediction: [[2.040119e-09]]
X: [203 205 118 203 205 124]
Prediction: [[3.944805e-09]]
X: [203 205 112 203 205 118]
Prediction: [[7.865198e-09]]
X: [203 205 106 203 205 112]
Prediction: [[1.396465e-08]]
X: [203 205 100 203 205 106]
Prediction: [[2.631063e-08]]
X: [203 129 130 203 205 100]
Prediction: [[6.6934662e-09]]
X: [203 133 134 203 129 130]
Prediction: [[1.7396316e-10]]
X: [203 137 138 203 133 134]
Prediction: [[1.0080783e-10]]
X: [203 141 142 203 137 138]
Prediction: [[5.8046287e-11]]
X: [203 143 144 203 141 142]
Prediction: [[3.8303957e-11]]
X: [203 146 146 203 143 144]
Prediction: [[2.952683e-11]]
X: [203 148 148 203 146 146]
Prediction: [[2.3433594e-11]]
X: [203 152 152 203 148 148]
Prediction: [[1.5621159e-11]]
X: [203 154 154 203 152 152]
Prediction: [[1.0568897e-11]]
X: [2

X: [203 133 134 203 131 132]
Prediction: [[1.5621912e-10]]
X: [203 136 136 203 133 134]
Prediction: [[1.2425072e-10]]
X: [203 138 138 203 136 136]
Prediction: [[1.00480145e-10]]
X: [203 142 142 203 138 138]
Prediction: [[6.16501e-11]]
X: [203 146 146 203 142 142]
Prediction: [[3.3480878e-11]]
X: [203 149 150 203 146 146]
Prediction: [[1.9216663e-11]]
X: [203 152 152 203 149 150]
Prediction: [[1.3713845e-11]]
X: [203 156 156 203 152 152]
Prediction: [[8.970374e-12]]
X: [203 160 160 203 156 156]
Prediction: [[4.9956515e-12]]
X: [203 163 164 203 160 160]
Prediction: [[2.6764133e-12]]
X: [203 166 166 203 163 164]
Prediction: [[1.9187828e-12]]
X: [203 169 170 203 166 166]
Prediction: [[1.2150807e-12]]
X: [203 173 174 203 169 170]
Prediction: [[6.8273436e-13]]
X: [203 175 176 203 173 174]
Prediction: [[4.6569454e-13]]
X: [203 177 178 203 175 176]
Prediction: [[3.561347e-13]]
X: [203 181 182 203 177 178]
Prediction: [[2.3064368e-13]]
X: [203 185 186 203 181 182]
Prediction: [[1.3723198e-13]]


1031/1031 [==============================] - 0s 28us/step - loss: 3.4534e-05 - accuracy: 1.0000
X: [109   0  22 203 205 206]
Prediction: [[1.2882099e-12]]
X: [118   0  16 109   0  22]
Prediction: [[0.00172408]]
X: [128   0  12 118   0  16]
Prediction: [[0.00181238]]
X: [139   0   8 128   0  12]
Prediction: [[0.00166161]]
X: [161   0   2 139   0   8]
Prediction: [[0.00161565]]
X: [184   0   0 161   0   2]
Prediction: [[0.00078067]]
X: [199   0   0 184   0   0]
Prediction: [[0.00030461]]
X: [201   0   0 199   0   0]
Prediction: [[0.00016423]]
X: [203   0   0 201   0   0]
Prediction: [[0.0001505]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [203   0   0 203   0   0]
Prediction: [[0.00013908]]
X: [20

X: [203 204 204 203 200 200]
Prediction: [[1.05587265e-14]]
X: [203 205 202 203 204 204]
Prediction: [[1.0977663e-14]]
X: [203 205 194 203 205 202]
Prediction: [[3.122751e-14]]
X: [203 205 188 203 205 194]
Prediction: [[9.700054e-14]]
X: [203 205 184 203 205 188]
Prediction: [[2.1524246e-13]]
X: [203 205 178 203 205 184]
Prediction: [[5.4029546e-13]]
X: [203 205 174 203 205 178]
Prediction: [[1.1777505e-12]]
X: [203 205 170 203 205 174]
Prediction: [[2.1909343e-12]]
X: [203 205 162 203 205 170]
Prediction: [[6.2042762e-12]]
X: [203 205 156 203 205 162]
Prediction: [[1.7312067e-11]]
X: [203 205 148 203 205 156]
Prediction: [[5.0465743e-11]]
X: [203 205 146 203 205 148]
Prediction: [[9.3078836e-11]]
X: [203 205 142 203 205 146]
Prediction: [[1.5071146e-10]]
X: [203 205 138 203 205 142]
Prediction: [[2.727013e-10]]
X: [203 205 134 203 205 138]
Prediction: [[4.891786e-10]]
X: [203 205 132 203 205 134]
Prediction: [[6.992487e-10]]
X: [203 205 128 203 205 132]
Prediction: [[1.0584381e-09]]
X

KeyboardInterrupt: 